In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import pandas as pd
from glob import glob
from pathlib import Path

import __init_path__
import env

from slam.linalg import RelativeTrajectory

In [3]:
prefixes = {
    1: '/dbstore/datasets/robotics/mlflow/odometry/artifacts/kitti_4_6/3d0b493b672a4407affe283e18ee7fbb/artifacts/flexible_baselene_s1_b_0/predictions',
    2: '/dbstore/datasets/robotics/mlflow/odometry/artifacts/kitti_4_6/f4162332bcd54f3080a5b92e7bdd7a6f/artifacts/flexible_baselene_s2_b_1/predictions',
    4: '/dbstore/datasets/robotics/mlflow/odometry/artifacts/kitti_4_6/b9cdbb7e4f564f8b873394b273fa905e/artifacts/flexible_baselene_s4_b_3/predictions',
    'loops': '/Vol0/user/d.zhukov/Projects/odometry/experiments/kitti_4_6_bovw/flexible_baseline_b_4/predictions'
}

In [4]:
def get_path(prefix, trajectory_name):
    paths = glob(f'{prefix}/*/val/*{trajectory_name}.csv')
    if len(paths) == 1:
        return paths[0]
    elif len(paths) > 1:
        return max(paths, key=lambda x: int(x[x.find('_val_RPE') - 3: x.find('_val_RPE')]))
    else:
        return glob(f'{prefix}/test/test/*{trajectory_name}.csv')[0]

In [5]:
def read_csv(path):
    df = pd.read_csv(path)
    df.rename(columns={'path_to_rgb': 'from_path',
                       'path_to_rgb_next': 'to_path'},
              inplace=True)
    
    stem_fn = lambda x: int(Path(x).stem)
    df['to_index'] = df['to_path'].apply(stem_fn)
    df['from_index'] = df['from_path'].apply(stem_fn)
    
    df['diff'] = df['to_index'] - df['from_index']
    
    mean_cols = ['euler_x', 'euler_y', 'euler_z', 't_x', 't_y', 't_z']
    std_cols = [c + '_confidence' for c in mean_cols]
    
    df = pd.concat((df, pd.DataFrame(columns=std_cols)), axis=1)
    df.fillna(1., inplace=True)
    return df


def get_predicted_df(paths):
    df_list = list()
    for stride, path in paths.items():
        df = read_csv(path)
        
        if stride == 'loops':
            df = df[df['diff'] > 49].reset_index()

        df_list.append(df)
    
    predicted_df = pd.concat(df_list, ignore_index=True)
    return predicted_df

In [6]:
def get_gt_trajectory(trajectory_name):
    gt_df = pd.read_csv(os.path.join(env.KITTI_MIXED_PATH, '1', trajectory_name, 'df.csv'))
    gt_trajectory = RelativeTrajectory.from_dataframe(gt_df).to_global()
    return gt_trajectory

In [7]:
trajectory_names = ['03', '04', '05', '06', '07', '10']

X = []
y = []
for trajectory_name in trajectory_names:
    paths = {stride: get_path(prefix, trajectory_name) for stride, prefix in prefixes.items()}
    
    predicted_df = get_predicted_df(paths)
    gt_trajectory = get_gt_trajectory(trajectory_name)
    
    X.append(predicted_df)
    y.append(gt_trajectory)

In [8]:
strides = [stride for stride in prefixes if stride != 'loops']

coefs = [
    [1, 1, 1],
    [1, 2, 4],
    [1, 6, 12],
    [1, 1, 1000000],
    [1, 1000000, 1000000]
]

param_distributions = {
    'coef': [dict(zip(strides, c)) for c in coefs],
    'coef_loop': [0, 1, 100, 300, 500], 
    'loop_threshold': [50, 100], 
    'rotation_scale': np.logspace(-10, 0, 11, base=2),
}

In [9]:
from scripts.aggregation import random_search

random_search(X, y, param_distributions, n_iter=1, verbose=1)

Using TensorFlow backend.


Fitting 1 folds for each of 1 candidates, totalling 1 fits
Running G2OEstimator(coef={1: 1, 2: 1, 4: 1000000}, coef_loop=300, loop_threshold=100,
       max_iterations=100, online=False, rotation_scale=0.001953125,
       verbose=False)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


	1: 1.0316956380441558
	2: 0.8926227967654983
	3: 4.230574507820375
	4: 2.92313583866514
	5: 2.511574314692217
	6: 2.907909388288141
	1: 1.0316956380441558
	2: 0.8926227967654983
	3: 4.230574507820375
	4: 2.92313583866514
	5: 2.511574314692217
	6: 2.907909388288141
Best params: {'rotation_scale': 0.001953125, 'loop_threshold': 100, 'coef_loop': 300, 'coef': {1: 1, 2: 1, 4: 1000000}}
Best score : 2.416252080712588


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  5.4min finished


{'mean_fit_time': array([0.]),
 'std_fit_time': array([0.]),
 'mean_score_time': array([160.247]),
 'std_score_time': array([0.]),
 'param_rotation_scale': masked_array(data=[0.001953125],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_loop_threshold': masked_array(data=[100],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_coef_loop': masked_array(data=[300],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_coef': masked_array(data=[{1: 1, 2: 1, 4: 1000000}],
              mask=[False],
        fill_value='?',
             dtype=object),
 'params': [{'rotation_scale': 0.001953125,
   'loop_threshold': 100,
   'coef_loop': 300,
   'coef': {1: 1, 2: 1, 4: 1000000}}],
 'split0_test_score': array([2.416]),
 'mean_test_score': array([2.416]),
 'std_test_score': array([0.]),
 'rank_test_score': array([1], dtype=int32),
 'split0_train_score': array([2.416]),
 'mean_train_sc